<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Блок-функций" data-toc-modified-id="Блок-функций-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Блок функций</a></span></li><li><span><a href="#Преобразование-данных" data-toc-modified-id="Преобразование-данных-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Преобразование данных</a></span></li><li><span><a href="#Подготовка-выборок-для-обучения" data-toc-modified-id="Подготовка-выборок-для-обучения-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Подготовка выборок для обучения</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Решающее-дерево" data-toc-modified-id="Решающее-дерево-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Решающее дерево</a></span></li><li><span><a href="#pipeline" data-toc-modified-id="pipeline-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>pipeline</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer()

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [4]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Загрузка данных

In [5]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
try:
    df_raw = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Яндекс_практикум/векторизация текста/toxic_comments.csv')
except:
    df_raw = pd.read_csv('/datasets/toxic_comments.csv')
df_raw.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
df_raw['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Как мы видим, имеется дисбаланс классов. Почти все комментарии (90%) отрицательные

### Блок функций

In [9]:
def get_clear_text(some_text):
    """с помощью регулярных выражений оставляем только символы алфавита"""
    some_text = some_text.lower()
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', some_text).split())
# ========================================================================
def get_tokens(some_text):
    """токенизация текста"""
    return word_tokenize(some_text)
# ========================================================================
def remove_stopwords(text):
    """убераем стоп-слова и токенизируем"""
    clean_text = [w for w in text.split() if not w in stop_words]  
    return clean_text
# ========================================================================
def get_lemm(some_text):
    lemma = ''
    for token in some_text:
        lemma = lemma + ' ' + lemmatizer.lemmatize(token)
    return lemma[1:]
# ========================================================================

### Преобразование данных

In [10]:
# очищаем текст
df_raw['clear'] = df_raw['text'].apply(get_clear_text)
df_raw.head()

,text,toxic,clear
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [11]:
# очищаем от стоп-слов, токенизируем
df_raw['unstop'] = df_raw['clear'].apply(remove_stopwords)
df_raw.head()

,text,toxic,clear,unstop
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, edits, made, username, hardcore,..."
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...,"[aww, matches, background, colour, seemingly, ..."
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...,"[hey, man, really, trying, edit, war, guy, con..."
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,"[make, real, suggestions, improvement, wondere..."
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,"[sir, hero, chance, remember, page]"


In [12]:
# лемматизируем
df_raw['lemm'] = df_raw['unstop'].apply(get_lemm)
df_raw.head()

,text,toxic,clear,unstop,lemm
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...,"[aww, matches, background, colour, seemingly, ...",aww match background colour seemingly stuck th...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...,"[make, real, suggestions, improvement, wondere...",make real suggestion improvement wondered sect...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,"[sir, hero, chance, remember, page]",sir hero chance remember page


In [13]:
# ради экономиии памяти удаляем ненужные столбцы
del df_raw['text']
del df_raw['clear']
del df_raw['unstop']

### Подготовка выборок для обучения

In [14]:
# выборка признаков и целевого признака
X = df_raw['lemm']
y = df_raw['toxic']

# удаляем датафрейм
del df_raw

In [15]:
try:
    X.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Яндекс_практикум/векторизация текста/X_raw.csv', index=False)
    y.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Яндекс_практикум/векторизация текста/y_raw.csv', index=False)
except:
    pass

In [16]:
state = 123

# разбиваем на обучающие и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=state,
    stratify=y
)

# проверяем
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# удаляем ради экономии памяти
del X
del y

(127656,)
(127656,)
(31915,)
(31915,)


In [17]:
# счётчик величин TF-IDF
tfidf = TfidfVectorizer(stop_words=stop_words)
tfidf.fit(X_train)

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [18]:
X_train = (tfidf.transform(X_train))

In [19]:
X_test = (tfidf.transform(X_test))

In [20]:
y_train = y_train.values

### Вывод:
- Произведена лемматизация текста
- Данные подготовлены для дальнейшего обучения моделей.

## Обучение

### Логистическая регрессия

Для борьбы с дисбалансом передадим модели аргумент class_weight='balanced'

In [21]:
model_linear = LogisticRegression(
    random_state=state,
    # class_weight='balanced',
    n_jobs=-1
    )

Подберем параметры с помощью кросс-валидации

In [22]:
skf = StratifiedKFold(
    n_splits=5,
    random_state=state,
    shuffle=True
)

params = {
    "C": np.arange(9, 12, 1),
    # "class_weight" : [None, 'balanced'],
    "max_iter": np.arange(28, 31, 1)
}

gscv = GridSearchCV(
    model_linear,
    params,
    scoring='f1',
    n_jobs=-1,
    cv=skf,
    verbose=1
    )

gscv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
             estimator=LogisticRegression(n_jobs=-1, random_state=123),
             n_jobs=-1,
             param_grid={'C': array([ 9, 10, 11]),
                         'max_iter': array([28, 29, 30])},
             scoring='f1', verbose=1)

In [23]:
print(gscv.best_params_)
print(gscv.best_score_)

{'C': 10, 'max_iter': 29}
0.7736158914238132


На кросс-валидации F1-score = 0.77. Посмотрим, какое значение метрики получится на тестовой выборке.

In [24]:
model_linear = LogisticRegression(
    random_state=state,
    n_jobs=-1,
    C=10,
    max_iter=29
    )

In [25]:
model_linear.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=29, n_jobs=-1, random_state=123)

In [26]:
predict_linear = model_linear.predict(X_test)
score_linear = f1_score(y_test, predict_linear)
score_linear.round(4)

0.7721

Значение F1-score больше 0,77. По условию проекта, данная модель годится для пользования клиентом.

### Решающее дерево

Рассмотрим еще одну модель. Подберем параметры на кросс-валидации

In [27]:
model_DTC = DecisionTreeClassifier(random_state=state)

In [28]:
params = {
    "max_depth": np.arange(36, 39, 1),
    'class_weight': ['balanced', None]
}

gscv = GridSearchCV(
    model_DTC,
    params,
    scoring='f1',
    n_jobs=-1,
    cv=skf,
    verbose=1
    )

gscv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=123), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': array([36, 37, 38])},
             scoring='f1', verbose=1)

In [29]:
print(gscv.best_params_)
print(gscv.best_score_)

{'class_weight': None, 'max_depth': 37}
0.689192864897839


In [30]:
model_DTC = DecisionTreeClassifier(
    random_state=state,
    max_depth=37
)

In [31]:
model_DTC.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=37, random_state=123)

In [32]:
predict_dtc = model_DTC.predict(X_test)
score_dtc = f1_score(y_test, predict_dtc)
score_dtc.round(4)

0.6846

Как мы видим, модель далека от намеченного результата

Дальнейшее обучение различных моделей не приведет к нужному результату и потратит время

### pipeline

In [ ]:
try:
    del X_train
    del X_test
    del predict_dtc
    del predict_linear
    del y_test
    del y_train
except:
    pass

In [ ]:
try:
    X = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Яндекс_практикум/векторизация текста/X_raw.csv')
    y = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Яндекс_практикум/векторизация текста/y_raw.csv')
except:
    pass

In [ ]:
X = X.values.astype('U')
y = y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=state,
    stratify=y
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

try:
    del X
    del y
except:
    pass

(127656, 1)
(127656, 1)
(31915, 1)
(31915, 1)


In [ ]:
model_linear = LogisticRegression(random_state=state, n_jobs=-1,)

pipe = make_pipeline(
    # CountVectorizer(),
    tfidf,
    model_linear
)

pipe

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...})),
                ('logisticregression',
                 LogisticRegression(n_jobs=-1, random_state=123))])

In [ ]:
params = {
    "logisticregression__C": np.arange(7, 10, 1),
    # "logisticregression__class_weight" : [None, 'balanced'],
    "logisticregression__max_iter": np.arange(20, 42, 10)
}

gscv = GridSearchCV(
    pipe,
    params,
    scoring='f1',
    n_jobs=-1,
    cv=skf,
    verbose=1
    )

gscv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


## Выводы

В работе:
- проведена лемматизация текстов,
- проведена подготовка признаков и целевых признаков,
- нами обучена модель логистической регрессии, предсказывающая оттенок комментария (негативный, позитивный),
- значение метрики F1 более 0.75, что удовлетворяет условию проекта.